# Extract Cycling Infrastructure from OSM
Run this notebook to get all of the potential bicycling infrastructure in the study area.


In [1]:
import geopandas as gpd
import pandas as pd
import ast
from pathlib import Path

from bikewaysim.paths import config, root

Import network

In [2]:
links = gpd.read_file(config['network_fp'] / f"osm.gpkg",layer='raw')
links0 = gpd.read_file(config['network_fp']/'networks.gpkg',layer='osm_links',ignore_geometry=True)
links = links[links['osmid'].isin(set(links0['osmid'].tolist()))]
link_type = dict(zip(links0['osmid'],links0['link_type']))
links['link_type'] = links['osmid'].map(link_type)
del links0

Modify manually identified features

In [3]:
# turns a feature into a multi use path
change_to_designated = [int(x) for x in (Path.cwd() / 'change_to_designated.txt').read_text().splitlines()]
# links.loc[links['osmid'].isin(change_to_designated)].explore()
#links.loc[links['osmid'].isin(change_to_designated)].to_file(config['bicycle_facilities_fp']/'scratch.gpkg',layer='change_to_designated')

In [4]:
# removes a feature from being considered a multi use path
exclude = [int(x) for x in (Path.cwd() / 'exclude_facilities.txt').read_text().splitlines()]
# links[links['osmid'].isin(exclude)].explore()
#links.loc[links['osmid'].isin(exclude)].to_file(config['bicycle_facilities_fp']/'scratch.gpkg',layer='exclude')

In [5]:
links.loc[links['osmid'].isin(change_to_designated),'bicycle'] = 'designated'
links = links[links['osmid'].isin(exclude)==False]

Get every feature with a cycleway related tag

In [6]:
links.drop(columns='cycleway',inplace=True)

# retrieve cycleway columns
all_cycleway_tags = {}
for idx, row in links.iterrows():
    #read the tags column as a dict
    tags = ast.literal_eval(row['all_tags'])
    #check for keys with cycleway mentioned
    cycleway_tags = {key:tags[key] for key in tags.keys() if "cycleway" in key}

    if len(cycleway_tags) > 0:
        all_cycleway_tags[idx] = cycleway_tags

#add as columns to the main dataframe
all_cycleway_tags_df = pd.DataFrame.from_dict(all_cycleway_tags,orient='index')

links = pd.merge(links, all_cycleway_tags_df, left_index=True, right_index=True, how='left')


is_cycleway = links['highway'].isin(['cycleway'])
cycleway_tags = (links['cycleway'].notna()) | (links.index.isin(all_cycleway_tags.keys()))
# NOTE removed permissive and permitted
peds_allowed = links['foot'].isin(['yes','designated'])
bikes_allowed = links['bicycle'].isin(['yes','designated'])

# reduce to features that are most likely to be cycleways
cycleways = links[is_cycleway | cycleway_tags | bikes_allowed].copy()

print('These are the cycleway tags used in the study area')
print(all_cycleway_tags_df.columns.tolist())

needed_cols = ['cycleway','cycleway:both','cycleway:right','cycleway:left']
for col in needed_cols:
    if col not in cycleways.columns.tolist():
        cycleways[col] = None

These are the cycleway tags used in the study area
['cycleway:both', 'cycleway:both:buffer', 'cycleway:right', 'cycleway:right:buffer', 'cycleway', 'cycleway:left', 'cycleway:left:buffer', 'cycleway:est_width', 'cycleway:both:lane', 'cycleway:crossing:markings']


# Classify Cycling Infrastructure Types
Heavily inspired from [People for Bikes](https://drive.google.com/file/d/1iJtlhDbTMEPdoUngrCKL-rfSK84ib081/view)

Cycling infrastructure was categorized into the following types by direction:
- Sharrow (Class III)
- Bicycle Lanes (Class II):
    - None (Class II)
    - Buffered (Class II)
    - Flex posts (Class II)
- Cycletracks or Bike Lanes with Physical Separation (Class IV)
- Multi-Use Trails (Class I)
    - Includes side paths and wide sidewalks
    - Multi-use trails/shared-use paths that don't follow a road

## Directional bike facilities

In [7]:
cycleways['facility_fwd'] = None #facility type for the forward direction (if any)
cycleways['facility_rev'] = None #facility type for the reverse direction (if any)

right_cols = cycleways.columns[cycleways.columns.str.startswith("cycleway:right")].tolist()
left_cols = cycleways.columns[cycleways.columns.str.startswith("cycleway:left")].tolist()

## No facility
Pre-assign ways that don't have a bicycle facility. If there is a seperate way (cycleway=seperate) then mark as no facility because there should be a corresponding cycletrack or multi-use path next to that way. Note if your study area does not have the requisite tags, black columns are added.


In [8]:
#mark all private access as no facility
cycleways.loc[cycleways['access'].isin(['no','private','customers']),['facility_fwd','facility_rev']] = 'no facility'

#mark any null or tags with no facility keywords as false
no_facility_keywords = ['no','separate','none']
not_cycleway = cycleways['link_type'] != 'bike' #cycleways['highway'].isin(['cycleway','path']) == False
not_both = cycleways[['cycleway','cycleway:both']].isna().all(axis=1) | cycleways[['cycleway','cycleway:both']].isin(no_facility_keywords).any(axis=1)
no_right = (cycleways['cycleway:right'].isna() | cycleways['cycleway:right'].isin(no_facility_keywords))
no_left = (cycleways['cycleway:left'].isna() | cycleways['cycleway:left'].isin(no_facility_keywords)) 

cycleways.loc[not_cycleway & not_both & no_right & cycleways['facility_fwd'].isna(),'facility_fwd'] = 'no facility'
cycleways.loc[not_cycleway & not_both & no_left & cycleways['facility_rev'].isna(),'facility_rev'] = 'no facility'

## Multi-use paths and cycletracks (Class I and Class IV)
OSM doesn't have a good way to distinguish between multi-use paths and cycletracks bike facilities. Sometimes a segregation tag will be used but many of the class iv bike facilities in Atlanta are bi-directional and thus are typically drawn as separate geometries. Because of this they look identical to side-paths and multi-use trails (class i). In the older data, there are a few occurances of highway=cycleway being accompanied by cycleway=lane. Cycleway should take precedent in these cases.

**Cycletracks (aka Class IV Bike Lanes) Should Have "foot = no" OR "foot IS NULL" AND be "highway=cycleway" but this isn't always consistent**

In [9]:
cycleways['highway'].unique()

array(['residential', 'tertiary', 'service', 'footway', 'secondary',
       'cycleway', 'pedestrian', 'trunk', 'path', 'primary', 'steps',
       'living_street'], dtype=object)


Cycletracks

In [10]:
no_peds = (peds_allowed==False) | cycleways['foot'].isna()
cycleways.loc[((cycleways['highway'].isin(['cycleway']) & no_peds)) & cycleways['facility_fwd'].isna(),'facility_fwd'] = 'cycletrack'
cycleways.loc[((cycleways['highway'].isin(['cycleway']) & no_peds)) & cycleways['facility_rev'].isna() & (cycleways['oneway']!='yes'),'facility_rev'] = 'cycletrack'

# NOTE: there are going to be cases in which there will be a bike facility on the left side of the road that's actually the forward direction facility
cycleways.loc[(cycleways['cycleway:right'] == 'track') & cycleways['facility_fwd'].isna(),'facility_fwd'] = 'cycletrack'
cycleways.loc[(cycleways['cycleway:left'] == 'track') & cycleways['facility_rev'].isna() & (cycleways['oneway']!='yes'),'facility_rev'] = 'cycletrack'

# for manual override
other_cycletracks = [179237451]
cycleways.loc[cycleways['osmid'].isin(other_cycletracks),'facility_fwd'] = 'cycletrack'
cycleways.loc[cycleways['osmid'].isin(other_cycletracks) & (cycleways['oneway']!='yes'),'facility_rev'] = 'cycletrack'

Multi-Use Paths

In [11]:
# anything in cycleway or path
cycleways.loc[(cycleways['highway'].isin(['cycleway'])) & cycleways['facility_fwd'].isna(),'facility_fwd'] = 'multi use path'
cycleways.loc[(cycleways['highway'].isin(['cycleway'])) & cycleways['facility_rev'].isna(),'facility_rev'] = 'multi use path'

# add links that are non motorized but still have a bike access tag
non_motorized = []
cycleways.loc[cycleways['link_type'].isin(['bike']) & cycleways['facility_fwd'].isna(),'facility_fwd'] = 'multi use path'
cycleways.loc[cycleways['link_type'].isin(['bike']) & cycleways['facility_rev'].isna(),'facility_rev'] = 'multi use path'

## Sharrows

In [12]:
#sharrows will have a shared_lane attribute value in the cycleway or cycleway:both column
sharrow = (cycleways[['cycleway','cycleway:both']] == 'shared_lane').any(axis=1)
cycleways.loc[sharrow & cycleways['facility_fwd'].isna(),'facility_fwd'] = 'sharrow'
cycleways.loc[sharrow & cycleways['facility_rev'].isna(),'facility_rev'] = 'sharrow'

#assume left = opposing direction and right = forward direction
sharrow_right = (cycleways["cycleway:right"] == 'shared_lane')
sharrow_left = (cycleways["cycleway:left"] == 'shared_lane')
cycleways.loc[sharrow_right & cycleways['facility_fwd'].isna(),'facility_fwd'] = 'sharrow'
cycleways.loc[sharrow_left & cycleways['facility_rev'].isna(),'facility_rev'] = 'sharrow'

## Buffered Bike Lanes

In [13]:
#buffered bike lanes
cycleways.loc[cycleways[right_cols+left_cols].isna().all(axis=1) & (cycleways['cycleway:both:buffer']=='yes') & cycleways['facility_fwd'].isna(),'facility_fwd'] = "buffered bike lane"
cycleways.loc[cycleways[right_cols+left_cols].isna().all(axis=1) & (cycleways['cycleway:both:buffer']=='yes') & cycleways['facility_rev'].isna(),'facility_rev'] = "buffered bike lane"

# left / assume to be the reverse direction
# buffered bike lanes
cycleways.loc[(cycleways['cycleway:left'] == 'lane') & (cycleways['cycleway:left:buffer'] == 'yes') & cycleways['facility_rev'].isna(),'facility_rev'] = 'buffered bike lane'

# right / assume to be the forward direction
# buffered bike lanes
cycleways.loc[(cycleways['cycleway:right'] == 'lane') & (cycleways['cycleway:right:buffer'] == 'yes') & cycleways['facility_fwd'].isna(),'facility_fwd'] = 'buffered bike lane'

## Traditional Bike Lanes

In [14]:
#traditional painted bike lanes (shouldn't have to worry about the buffered part now)
no_left_right_attrs = (cycleways[right_cols+left_cols].isna().all(axis=1)) | (cycleways[right_cols+left_cols] == 'no').any(axis=1)
cycleways.loc[
    no_left_right_attrs & (cycleways[['cycleway:both','cycleway']].isin(['lane','yes'])).any(axis=1) & cycleways['facility_fwd'].isna(),'facility_fwd'] = "bike lane"
cycleways.loc[
    no_left_right_attrs & (cycleways[['cycleway:both','cycleway']].isin(['lane','yes'])).any(axis=1) & cycleways['facility_rev'].isna(),'facility_rev'] = "bike lane"

#left bike lanes
cycleways.loc[(cycleways['cycleway:left'].isin(['lane','yes'])) & cycleways['facility_rev'].isna(),'facility_rev'] = 'bike lane'
cycleways.loc[(cycleways['cycleway:left'] == 'opposite_lane') & cycleways['facility_rev'].isna(),'facility_rev'] = 'bike lane' #'contra flow bike lane'

#right bike lanes
cycleways.loc[(cycleways['cycleway:right'].isin(['lane','yes'])) & cycleways['facility_fwd'].isna(),'facility_fwd'] = 'bike lane'
cycleways.loc[(cycleways['cycleway:right'] == 'opposite_lane') & cycleways['facility_fwd'].isna(),'facility_fwd'] = 'bike lane'

Check: what's still unclassified?

In [15]:
unclassifed = cycleways[cycleways['facility_fwd'].isna() | cycleways['facility_rev'].isna()]
unclassifed
#unclassifed.drop(columns='all_tags').explore()

,osmid,timestamp,version,type,highway,oneway,name,bridge,tunnel,service,...,cycleway:right,cycleway:right:buffer,cycleway,cycleway:left,cycleway:left:buffer,cycleway:est_width,cycleway:both:lane,cycleway:crossing:markings,facility_fwd,facility_rev
2231,169927468,1727109795,9,way,tertiary,None,Marietta Street,None,None,None,...,NaN,NaN,track,NaN,NaN,NaN,NaN,NaN,None,None
16007,1387354961,1749315409,3,way,footway,None,Westside Beltline Connector,None,None,None,...,NaN,NaN,crossing,NaN,NaN,NaN,NaN,NaN,None,None


# Drop no facility options

In [16]:
no_facility = (cycleways[['facility_fwd','facility_rev']] == 'no facility').all(axis=1)
cycleways = cycleways[no_facility==False]

# Remove Dirt Trails and Hiking Trails (included in routing but will have to mess around with speed)
- highway=path but bicycle=no or null
- OR surface=dirt/sand/unpaved (etc)

In [17]:
#filter out some of the dirt trails
def get_surface_tag(item):
    tags = ast.literal_eval(item)
    surface_tag = tags.get('surface',0)
    if surface_tag != 0:
        return surface_tag
    else:
        return None
cycleways['surface'] = cycleways['all_tags'].apply(get_surface_tag)

#all_cycleways['surface'].unique()
remove = ['gravel','log','wood','ground', 'grass', 'unpaved', 'dirt',
       'mud', 'stepping_stones', 'fine_gravel', 'brick', 'dirt/sand']
cycleways = cycleways[cycleways['surface'].isin(remove)==False]

#remove if bike is not allowed
bike_not_allowed = ['no','private','unkwown']
cycleways = cycleways[cycleways['bicycle'].isin(bike_not_allowed)==False]

#remove if highway=path and bike is na 
cycleways = cycleways[((cycleways['highway']=='path') & (cycleways['bicycle'].isna()))==False]

# Identify the cycletracks

In [18]:
# excl = gpd.read_file(Path("D:/PROJECTS/GDOT/GDOT/Bicycle_Facilities/scratch.gpkg"),layer='exclude')
# excl.osmid.tolist()

In [19]:

exclude = [
 ]
cycleways = cycleways[cycleways['osmid'].isin(exclude)==False]

# Export

In [20]:
final_cycleways = cycleways[~cycleways['facility_fwd'].isna() & ~cycleways['facility_rev'].isna()]
final_cycleways
#create both directions column for simpliciity 

,osmid,timestamp,version,type,highway,oneway,name,bridge,tunnel,service,...,cycleway:right,cycleway:right:buffer,cycleway,cycleway:left,cycleway:left:buffer,cycleway:est_width,cycleway:both:lane,cycleway:crossing:markings,facility_fwd,facility_rev
32,9190979,1746971668,12,way,tertiary,None,Johnson Road Northeast,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bike lane,bike lane
88,9240245,1588704387,4,way,footway,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,multi use path,multi use path
89,9240373,1699560896,15,way,residential,yes,State Street Northwest,None,None,None,...,lane,no,NaN,NaN,NaN,NaN,NaN,NaN,bike lane,no facility
94,9240894,1737654223,29,way,tertiary,None,Ferst Drive Northwest,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bike lane,bike lane
123,9243351,1748237083,11,way,residential,None,Fowler Street Northwest,None,None,None,...,NaN,NaN,shared_lane,NaN,NaN,NaN,NaN,NaN,sharrow,sharrow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18632,1423508867,1755679714,1,way,residential,None,Montgomery Ferry Drive Northeast,None,None,None,...,NaN,NaN,shared_lane,NaN,NaN,NaN,NaN,NaN,sharrow,sharrow
18639,177014164,1617322651,5,way,pedestrian,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,multi use path,multi use path
18643,667730432,1747892619,7,way,pedestrian,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,multi use path,multi use path
18644,667730439,1730262961,7,way,pedestrian,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,multi use path,multi use path


In [21]:
final_cycleways[['facility_fwd','facility_rev']].value_counts()

facility_fwd        facility_rev      
multi use path      multi use path        332
bike lane           bike lane              76
sharrow             sharrow                73
bike lane           no facility            63
cycletrack          cycletrack             39
buffered bike lane  buffered bike lane     36
sharrow             no facility            15
cycletrack          no facility             5
bike lane           sharrow                 4
cycletrack          multi use path          4
buffered bike lane  bike lane               3
no facility         bike lane               3
bike lane           buffered bike lane      1
cycletrack          bike lane               1
sharrow             bike lane               1
Name: count, dtype: int64

In [22]:
order = {
    'sharrow': 0,
    'bike lane': 1,
    'buffered bike lane': 2,
    'cycletrack': 3,
    'multi use path': 4
}
rev_order = {item:key for key,item in order.items()}
import numpy as np
array = pd.concat([final_cycleways['facility_fwd'].map(order),final_cycleways['facility_rev'].map(order)],axis=1)
final_cycleways['facility'] = array.max(axis=1).map(rev_order)

/opt/miniconda3/envs/bikewaysim/lib/python3.13/site-packages/geopandas/geodataframe.py:1968: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [23]:
final_cycleways['facility'].value_counts()

facility
multi use path        336
bike lane             147
sharrow                88
cycletrack             45
buffered bike lane     40
Name: count, dtype: int64

In [24]:
# NOTE nevermind these probaly aren't going to impact much
## Remove cycletrack connnectors
# These are really short features tagged as cycleways that serve to connect cycletracks/mups back to the road network. These should still be included for routing purposes but I wouldn't call them infrastructure. After experimenting, 100 feet seemed like a good threshold to capture these.?
# final_cycleways[(final_cycleways['facility']=='cycletrack') & (final_cycleways['highway']=='cycleway') & (final_cycleways.length < 100)].explore()

In [25]:
final_cycleways[final_cycleways['highway']=='path']['surface'].unique()

array(['concrete', 'paved', None, 'asphalt', 'paving_stones'],
      dtype=object)

In [26]:
#remove sharrows for now
final_cycleways = final_cycleways[final_cycleways['facility']!='sharrow']

final_cycleways.loc[final_cycleways['facility_fwd']=='no facility','facility_fwd'] = None
final_cycleways.loc[final_cycleways['facility_rev']=='no facility','facility_rev'] = None
final_cycleways.loc[final_cycleways['facility']=='no facility','facility'] = None

In [27]:
cols_needed = ['osmid','name','all_tags','facility_fwd','facility_rev','facility']
final_cycleways.to_file(config['bicycle_facilities_fp']/'reference_layers.gpkg',layer='osm_cycleways_full')

osm_network = gpd.read_file(config['network_fp']/'networks.gpkg',layer='osm_links')
merged = pd.merge(osm_network,final_cycleways[cols_needed],on='osmid')
merged.to_file(config['bicycle_facilities_fp']/'reference_layers.gpkg',layer='osm_cycleways_network')